In [429]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [430]:
# Read data from file
with open('data.txt', 'r') as f:
    data = f.readlines()

# Inputs
defense_ranks = np.array([int(i) for i in data[0].split(',')], dtype=np.float32)
skill_score = np.array([14.64 for _ in range(len(defense_ranks))], dtype=np.float32)

# Targets
fantasy_scores = np.array([float(i) for i in data[1].split(',')], dtype=np.float32)

# Training data
inputs = torch.cat((torch.from_numpy(defense_ranks[:10]).view(-1, 1).float(),
                          torch.from_numpy(skill_score[:10]).view(-1, 1).float()), dim=1)
targets = torch.from_numpy(fantasy_scores[:10]).view(-1, 1).float()

# Testing data
test_inputs = torch.cat((torch.from_numpy(defense_ranks[10:]).view(-1, 1).float(),
                         torch.from_numpy(skill_score[10:]).view(-1, 1).float()), dim=1)
test_targets = torch.from_numpy(fantasy_scores[10:]).view(-1, 1).float()


In [432]:
# Define the neural network architecture
class RegressionModel(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

# Set hyperparameters
input_size = 2
hidden_size = 30
learning_rate = 0.001
num_epochs = 100

# Create the model
model = RegressionModel(input_size, hidden_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [447]:
# Training loop
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(inputs)
    loss = criterion(outputs, targets)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss for every epoch
    # print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Make predictions
with torch.no_grad():
    predicted = model(test_inputs)

print(f'loss = {loss.item()}\n') # add blank line
# Print the predictions and original scores
for i in range(len(test_inputs)):
    print(f"Defense Rank: {test_inputs[i][0]}, Predicted Fantasy Score: {predicted[i][0]}, Actual Fantasy Score: {test_targets[i].item()}")

loss = 18.989673614501953

Defense Rank: 1.0, Predicted Fantasy Score: 6.102067947387695, Actual Fantasy Score: 5.599999904632568
Defense Rank: 10.0, Predicted Fantasy Score: 10.42695426940918, Actual Fantasy Score: 14.300000190734863
Defense Rank: 29.0, Predicted Fantasy Score: 18.36905860900879, Actual Fantasy Score: 7.599999904632568
Defense Rank: 15.0, Predicted Fantasy Score: 9.510947227478027, Actual Fantasy Score: 23.0
Defense Rank: 18.0, Predicted Fantasy Score: 10.619852066040039, Actual Fantasy Score: 7.699999809265137
